
Тестовий пакет – MIRFlickr-20k (https://press.liacs.nl/mirflickr/#sec_download)
Вибірка зображень – 250 зображень;
Формування вибірки зображень – псевдовипадкове, з використанням генератора Мерсена (стартове значення співпадає з номером студента в загальному списку групи) за модулем кількості зображень в тестовому пакеті.

#### Лабораторна робота №2
	Сформувати тестову вибірку зображень з вихідного пакету;

In [1]:
import numpy as np

import os
import cv2
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt


os.chdir('C:\\DS\\datasets\\mirflickr')# all datasets will be stored outside of a project
total_students = 10
student_num = 3%total_students

To select a sample we'll use numpy.random module, an implementation of Mersenne Twister pseudorandom generator based on [MT19937 algorithm](https://en.wikipedia.org/wiki/Mersenne_Twister). It's worth mentioning that python's random module is identical

In [2]:
contents = os.listdir()
images_names = np.array([c for c in contents if c.endswith('.jpg')])

# np.seed(student_num)
generator = np.random.RandomState(seed=student_num)# works the same as np.seed()

total_images = images_names.shape[0]
sample_size = 250

sample_images_names = generator.choice(images_names,sample_size) 

print(f'selected sample {sample_size} of images from {total_images} images in total')

selected sample 250 of images from 25000 images in total


To read the images we'll use opencv, which by default reads image in B,G,R order. 
* cv.imread(file) reads image as an ndarray of shape (height,width,3)
* cv.split() split ndarray into three 2D arrays (B,G,R) of shape (height,width), from which we select array G

In [3]:
#sample_images = np.array([cv2.imread(img) for img in sample_images_names])
sample_images_green = np.array([cv2.split(cv2.imread(img))[1] for img in sample_images_names])
green_pixels_list = np.concatenate([img.flatten() for img in sample_images_green])

* 1) 	Провести декомпозицію каналу зеленого кольору тестових зображень з використанням методу головних компонент (PCA):

In [4]:
from sklearn.decomposition import PCA
PCA(sample_images_green)

PCA(copy=True, iterated_power='auto',
    n_components=array([array([[ 47,  45,  44, ..., 116, 115, 114],
       [ 46,  44,  43, ..., 115, 114, 114],
       [ 44,  42,  41, ..., 116, 116, 116],
       ...,
       [ 71,  72,  74, ..., 126, 127, 130],
       [ 66,  67,  68, ..., 130, 132, 132],
       [ 64,  64,  64, ..., 130, 132, 132]], dtype=uint8),
       array([[ 3,  3,  4, ..., 27, 26, 25],
       [ 3,  3,  3, ..., 27, 26, 26],
       [ 3,  3,  3, ..., 27, 26, 24],
       ...,
       [74, 73, 73, ...,  9,  6,  5],
       [72, 72, 72, .....
       [119, 134, 145, ..., 145, 140, 131],
       [141, 124, 131, ..., 152, 139, 124],
       ...,
       [ 39,  32,  38, ...,  67,  64,  60],
       [ 59,  62,  58, ...,  60,  60,  58],
       [ 54,  46,  47, ...,  53,  51,  51]], dtype=uint8),
       array([[ 1,  2,  3, ...,  2,  2,  3],
       [ 3,  2,  4, ...,  2,  2,  2],
       [ 0,  2,  5, ...,  2,  2,  2],
       ...,
       [ 7,  6, 15, ...,  2,  2,  1],
       [ 5,  5,  9, ...,  2,  1,

2) 	Для каналу зеленого кольору тестових зображень обчислити наступні характеристики розподілу значень яскравості пікселів:
*	Впорядкувати отримані компоненти вихідного зображення в порядку зменшення значень сингулярних чисел (від найбільшого $s_{max}$ до найменшого $s_{min}$ значення);


*	Провести реконструкцію зображення при використанні лише частки (α%) компонентів розкладу, що характеризуються відмінними від нуля сингулярними числами ($s_i≠0$). Розглянути випадок, коли α змінюється від $α_{min}=10\%$ до $α_{max}=100\%$ з кроком $∆_α=10\%$

*	Порівняти вихідне та реконструйоване зображення за показником середньоквадратичного відхилення (MSE). 

*	Побудувати графіки залежності ($\overline {MSE}$(α)), де $\overline {MSE}$ - значення середньо-квадратичного відхилення між вихідним та реконструйованим зображеннями, усереднені по тестовому пакету;

* 4) 	Провести моделювання каналу зеленого кольору тестових зображень з використанням марківських ланцюгів першого порядку $M_1$:
    *	Отримати стохастичну матрицю для каналу зеленого кольору при обробці пікселів (згідно номеру студента в списку групи, за модулем кількості варіантів):
        *	По горизонталі, зліва направо – M_1^→ (I_(x,y),I_(x+1,y) );
        *	По горизонталі, справа наліво – M_1^← (I_(x,y),I_(x-1,y) );
        *	По вертикалі, зверху вниз – M_1^↓ (I_(x,y),I_(x,y+1) );
        *	По вертикалі, знизу вгору – M_1^↑ (I_(x,y),I_(x,y-1) );
        *	По головній діагоналі – M_1^↘ (I_(x,y),I_(x+1,y+1) );
        *	По головній діагоналі – M_1^↖ (I_(x,y),I_(x-1,y-1) );
        *	По побічній діагоналі – M_1^↙ (I_(x,y),I_(x-1,y+1) );
        *	По побічній діагоналі – M_1^↗ (I_(x,y),I_(x+1,y-1) );
*	В протоколі роботи графічно показати вид марківського ланцюга для діапазону яскравості пікселів I_(x,y)∈[i;i×10], де i- номеру студента в списку групи;
*	Для отриманих марківських ланцюгів перевірити виконання властивості регулярності та рекурентності після проходження 5 ітерацій роботи.

